# Problem Statement 2

An orthopaedist is a medical doctor specializing in diagnosing and treating disorders related to the skeletal system. Part of their job is to distinguish between a healthy person and a person with Osteoarthritis by looking at their knee X-ray images.

**Osteoarthritis :**  Osteoarthritis commonly known as wear-and-tear arthritis, is a condition in which the natural cushioning between joints — cartilage — wears away. When this happens, the bones of the joints rub more closely against one another with less of the shock-absorbing benefits of cartilage. The rubbing results in pain, swelling, stiffness, decreased ability to move, and, sometimes, the formation of bone spurs.


![alt text](https://lh5.googleusercontent.com/rYigiybgSG_wKaBc25YGO-wa8BS7h4IrP-aYu3zKEWJBQ4-fX-5gIq7qZiQ5CCaO0yD2Ea93S4n7duv_C01wboanM-SiPZjkJdlsTNRBSi_1ybijBq-CevrJf_XdHFBj-e8Gx0DM)
<center><b>Healthy Knee Joint</b></center>


![alt text](https://lh5.googleusercontent.com/uWkeoxyr1CsTwRH2Hkwruwe9-5JtNImXB7206usWlPICKpETxGQ1Ok_U8gjqz_oN5azb9Yzjxj9jYmlP0Fhxq8lt53TEL_-KtJE7l8In1kjDajZkLkIfPTC8x4Y7K_ztuqCyTShh)
<center><b>Osteoarthritis Condition</b></center>




I have created a deep learning model that can detect if osteoarthritis is present or not in a given knee X-ray image.


### Diractory Strurcture

- Dataset 

  The Dataset contains three folders 

  - Test (845 images)
  - Train (2350 images)
  - Valid (641 images)


- IPython Notebook

In [1]:
# Importing all necessary libraries
from glob import glob
import os
import random
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model, Model, Sequential

In [2]:
IMG_height = 224
IMG_width = 224

# Sprcifying directory path
train_path = './Dataset/train'
val_path = './Dataset/valid'
test_path = './Dataset/test'

In [3]:
# here name of classes == name of dirctories
classes = os.listdir(train_path)

# Counting total number of images for training and valdation
num_train = len(glob(train_path + '/*/*'))
num_val = len(glob(val_path + '/*/*'))
print('Classes : ', classes)
print(f'Number of training images : {num_train}\nNumber of validation images : {num_val}')

Classes :  ['Normal', 'Osteoarthritis']
Number of training images : 2350
Number of validation images : 641


In [4]:
# Loading images from image paths
def parse_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, [IMG_height, IMG_width])
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [5]:
def make_dataset(path, batch_size):
    
    # Collecting all filepath in a directory
    filenames = glob(path + '/*/*')
    # Shuffling the dataset
    random.shuffle(filenames)
    # Extraction labels from fil paths(as numbers Normal:0, Osteoarthritis:1)
    labels = [classes.index(name.split(os.path.sep)[-2]) for name in filenames]

    # Creating instance of tf.data.dataset from filenames
    filenames_ds = tf.data.Dataset.from_tensor_slices(filenames)
    # loading files
    images_ds = filenames_ds.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    images_ds = tf.data.Dataset.zip((images_ds, labels_ds))
    images_ds = images_ds.shuffle(buffer_size=100)
    images_ds = images_ds.batch(batch_size)
    images_ds = images_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return images_ds

In [6]:
%%time
ds_train = make_dataset(train_path, 32)
ds_val = make_dataset(train_path, 32)
ds_test = make_dataset(train_path, 128)

Wall time: 1.11 s


In [7]:
callbacks = [
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=5,
        verbose=1,
    )
]

In [8]:
# defining model
def build_model_A():
    batch_size = 32
    num_classes = 2

    i = Input(shape=(224, 224, 1))
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dropout(0.2)(x)

    # Hidden layer
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)

    # last hidden layer i.e.. output layer
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(i, x)

    # model description
    model.summary()

    model.compile(optimizer=keras.optimizers.Adam(3e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [9]:
model = build_model_A()
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 224, 224, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0     

In [10]:
# training model
model.fit(ds_train, epochs=20, steps_per_epoch=math.ceil(num_train/32), 
          verbose=1, callbacks=callbacks, validation_data=ds_val,
         validation_steps=math.ceil(num_val/32))

Epoch 1/20
74/74 [==============================] - 38s 464ms/step - loss: 2.4988 - accuracy: 0.7285 - val_loss: 28.1407 - val_accuracy: 0.3333
Epoch 2/20
74/74 [==============================] - 34s 465ms/step - loss: 0.3560 - accuracy: 0.8438 - val_loss: 25.5515 - val_accuracy: 0.3348
Epoch 3/20
74/74 [==============================] - 38s 514ms/step - loss: 0.2792 - accuracy: 0.8753 - val_loss: 4.3747 - val_accuracy: 0.3348
Epoch 4/20
74/74 [==============================] - 59s 794ms/step - loss: 0.1973 - accuracy: 0.9179 - val_loss: 3.1106 - val_accuracy: 0.5298
Epoch 5/20
74/74 [==============================] - 55s 744ms/step - loss: 0.1838 - accuracy: 0.9209 - val_loss: 2.2018 - val_accuracy: 0.5938
Epoch 6/20
74/74 [==============================] - 47s 628ms/step - loss: 0.1446 - accuracy: 0.9438 - val_loss: 2.7259 - val_accuracy: 0.6652
Epoch 7/20
74/74 [==============================] - 47s 629ms/step - loss: 0.1217 - accuracy: 0.9506 - val_loss: 1.4255 - val_accuracy: 0.70

In [11]:
# evaluating model
model.evaluate(ds_test)

19/19 [==============================] - 28s 773ms/step - loss: 0.0727 - accuracy: 0.9711


[0.07271134853363037, 0.9710638523101807]

In [13]:
# saving model
model.save('Osteoarthritis_Classifier.model', save_format='h5')